In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import lightgbm as lgb
import catboost as cbt

from sklearn import metrics
from sklearn.model_selection import StratifiedKFold, KFold, RepeatedKFold
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.preprocessing import StandardScaler as std

pd.set_option('display.max_columns', None)

In [11]:
path='./input/'
train = pd.read_csv(path + 'happiness_train_complete.csv', encoding='gbk')
test = pd.read_csv(path + 'happiness_test_complete.csv', encoding='gbk')
train.shape, test.shape

((8000, 140), (2968, 139))

In [13]:
for i in train.columns:
    if i not in test.columns:
        print(i)
    

happiness


In [14]:
features = [i for i in train.columns if i not in ['id', 'happiness']]
cat_col = [i for i in train.select_dtypes('object').columns if i not in ['id', 'happiness']]
len(features), len(cat_col)

(138, 4)

In [17]:
cat_col

['survey_time', 'edu_other', 'property_other', 'invest_other']

In [19]:
train.head()

,id,happiness,survey_type,province,city,county,survey_time,gender,birth,nationality,religion,religion_freq,edu,edu_other,edu_status,edu_yr,income,political,join_party,floor_area,property_0,property_1,property_2,property_3,property_4,property_5,property_6,property_7,property_8,property_other,height_cm,weight_jin,health,health_problem,depression,hukou,hukou_loc,media_1,media_2,media_3,media_4,media_5,media_6,leisure_1,leisure_2,leisure_3,leisure_4,leisure_5,leisure_6,leisure_7,leisure_8,leisure_9,leisure_10,leisure_11,leisure_12,socialize,relax,learn,social_neighbor,social_friend,socia_outing,equity,class,class_10_before,class_10_after,class_14,work_exper,work_status,work_yr,work_type,work_manage,insur_1,insur_2,insur_3,insur_4,family_income,family_m,family_status,house,car,invest_0,invest_1,invest_2,invest_3,invest_4,invest_5,invest_6,invest_7,invest_8,invest_other,son,daughter,minor_child,marital,marital_1st,s_birth,marital_now,s_edu,s_political,s_hukou,s_income,s_work_exper,s_work_status,s_work_type,f_birth,f_edu,f_political,f_work_14,m_birth,m_edu,m_political,m_work_14,status_peer,status_3_before,view,inc_ability,inc_exp,trust_1,trust_2,trust_3,trust_4,trust_5,trust_6,trust_7,trust_8,trust_9,trust_10,trust_11,trust_12,trust_13,neighbor_familiarity,public_service_1,public_service_2,public_service_3,public_service_4,public_service_5,public_service_6,public_service_7,public_service_8,public_service_9
0,1,4,1,12,32,59,2015/8/4 14:18,1,1959,1,1,1,11,NaN,4.0,-2.0,20000,1,NaN,45.0,0,1,0,0,0,0,0,0,0,NaN,176,155,3,2,5,5,2.0,4,2,5,5,4,3,1,4,3,1,2,3,4,1,4,5,4,1,2,4,3,3.0,3.0,2,3,3,3,3,1,1,3.0,30.0,1.0,2.0,1,1,1,2,60000.0,2,2,1,2,0,1,0,0,0,0,0,0,0,NaN,1,0,0.0,3,1984.0,1958.0,1984.0,6.0,1.0,5.0,40000.0,5.0,NaN,NaN,-2,4,4,1,-2,4,1,1,3,2,4,3,50000.0,4,2,-8,-8,5,3,2,3,4,3,-8,4,1,4,50,60,50,50,30.0,30,50,50,50
1,2,4,2,18,52,85,2015/7/21 15:04,1,1992,1,1,1,12,NaN,4.0,2013.0,20000,1,NaN,110.0,0,0,0,0,1,0,0,0,0,NaN,170,110,5,4,3,1,1.0,2,2,1,3,5,1,2,3,4,3,5,4,3,2,3,4,5,1,2,4,3,6.0,2.0,1,3,6,4,8,5,1,3.0,2.0,1.0,3.0,1,1,1,1,40000.0,3,4,1,2,0,1,0,0,0,0,0,0,0,NaN,0,0,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1972,3,1,2,1973,3,1,2,1,1,4,2,50000.0,5,4,4,3,5,3,3,3,2,3,3,3,2,3,90,70,70,80,85.0,70,90,60,60
2,3,4,2,29,83,126,2015/7/21 13:24,2,1967,1,0,3,4,NaN,4.0,-2.0,2000,1,NaN,120.0,0,1,1,0,0,0,0,0,0,NaN,160,122,4,4,5,1,1.0,2,2,2,5,1,3,1,4,4,3,5,4,4,2,3,5,5,5,3,4,2,2.0,5.0,2,4,5,4,6,3,2,NaN,NaN,NaN,NaN,1,1,2,2,8000.0,3,3,1,2,0,1,0,0,0,0,0,0,0,NaN,0,2,1.0,3,1990.0,1968.0,1990.0,3.0,1.0,1.0,6000.0,3.0,NaN,NaN,-2,1,1,2,-2,1,1,2,2,1,4,2,80000.0,3,3,3,3,4,3,3,3,3,3,-8,3,1,4,90,80,75,79,80.0,90,90,90,75
3,4,5,2,10,28,51,2015/7/25 17:33,2,1943,1,1,1,3,NaN,4.0,1959.0,6420,1,NaN,78.0,0,0,0,1,0,0,0,0,0,NaN,163,170,4,4,4,1,2.0,2,1,1,5,1,1,1,5,2,4,5,4,5,1,1,5,5,5,2,4,4,1.0,6.0,1,4,5,5,7,2,4,NaN,NaN,NaN,NaN,2,2,2,2,12000.0,3,3,1,1,0,1,0,0,0,0,0,0,0,NaN,1,4,0.0,7,1960.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-2,14,1,2,-2,1,1,2,2,1,3,2,10000.0,3,3,4,3,5,3,3,5,4,3,3,3,2,3,100,90,70,80,80.0,90,90,80,80
4,5,4,1,7,18,36,2015/8/10 9:50,2,1994,1,1,1,12,NaN,1.0,2014.0,-1,2,NaN,70.0,0,0,0,0,1,0,0,0,0,NaN,165,110,5,5,3,2,3.0,1,3,4,2,5,5,3,3,3,2,4,4,3,5,2,5,5,1,4,3,4,7.0,5.0,3,2,1,1,1,4,6,NaN,NaN,NaN,NaN,1,2,2,2,-2.0,4,3,1,1,0,1,0,0,0,0,0,0,0,NaN,0,0,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1970,6,1,10,1972,4,1,15,3,2,3,-8,200000.0,4,3,3,3,5,5,3,4,3,3,3,3,2,2,50,50,50,50,50.0,50,50,50,50


In [31]:
test['happiness'] = -1
data = train.append(test).reset_index(drop=True)
data.shape

D:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:7138: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort,


(10968, 140)

In [32]:
for i in features:
    if data[i].isnull().sum() != 0:
        if i in cat_col:
            data[i] = data[i].fillna('None')
        else:
            data[i] = data[i].fillna(0)

In [33]:
data.happiness.value_counts()

 4    4818
-1    2968
 5    1410
 3    1159
 2     497
 1     104
-8      12
Name: happiness, dtype: int64

In [34]:
data.head()

,birth,car,city,class,class_10_after,class_10_before,class_14,county,daughter,depression,edu,edu_other,edu_status,edu_yr,equity,f_birth,f_edu,f_political,f_work_14,family_income,family_m,family_status,floor_area,gender,happiness,health,health_problem,height_cm,house,hukou,hukou_loc,id,inc_ability,inc_exp,income,insur_1,insur_2,insur_3,insur_4,invest_0,invest_1,invest_2,invest_3,invest_4,invest_5,invest_6,invest_7,invest_8,invest_other,join_party,learn,leisure_1,leisure_10,leisure_11,leisure_12,leisure_2,leisure_3,leisure_4,leisure_5,leisure_6,leisure_7,leisure_8,leisure_9,m_birth,m_edu,m_political,m_work_14,marital,marital_1st,marital_now,media_1,media_2,media_3,media_4,media_5,media_6,minor_child,nationality,neighbor_familiarity,political,property_0,property_1,property_2,property_3,property_4,property_5,property_6,property_7,property_8,property_other,province,public_service_1,public_service_2,public_service_3,public_service_4,public_service_5,public_service_6,public_service_7,public_service_8,public_service_9,relax,religion,religion_freq,s_birth,s_edu,s_hukou,s_income,s_political,s_work_exper,s_work_status,s_work_type,socia_outing,social_friend,social_neighbor,socialize,son,status_3_before,status_peer,survey_time,survey_type,trust_1,trust_10,trust_11,trust_12,trust_13,trust_2,trust_3,trust_4,trust_5,trust_6,trust_7,trust_8,trust_9,view,weight_jin,work_exper,work_manage,work_status,work_type,work_yr
0,1959,2,32,3,3,3,1,59,0,5,11,None,4.0,-2.0,3,-2,4,4,1,60000.0,2,2,45.0,1,4,3,2,176,1,5,2.0,1,3,50000.0,20000,1,1,1,2,0,1,0,0,0,0,0,0,0,None,0.0,3,1,5,4,1,4,3,1,2,3,4,1,4,-2,4,1,1,3,1984.0,1984.0,4,2,5,5,4,3,0.0,1,4,1,0,1,0,0,0,0,0,0,0,None,12,50,60.0,50,50,30.0,30,50,50,50,4,1,1,1958.0,6.0,5.0,40000.0,1.0,5.0,0.0,0.0,2,3.0,3.0,2,1,2,3,2015/8/4 14:18,1,4,3,-8,4,1,2,-8,-8,5,3,2,3,4,4,155,1,2.0,3.0,1.0,30.0
1,1992,2,52,6,8,4,5,85,0,3,12,None,4.0,2013.0,3,1972,3,1,2,40000.0,3,4,110.0,1,4,5,4,170,1,1,1.0,2,2,50000.0,20000,1,1,1,1,0,1,0,0,0,0,0,0,0,None,0.0,3,2,4,5,1,3,4,3,5,4,3,2,3,1973,3,1,2,1,0.0,0.0,2,2,1,3,5,1,0.0,1,3,1,0,0,0,0,1,0,0,0,0,None,18,90,70.0,70,80,85.0,70,90,60,60,4,1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,2.0,6.0,2,0,1,1,2015/7/21 15:04,2,5,3,3,3,2,4,4,3,5,3,3,3,2,4,110,1,3.0,3.0,1.0,2.0
2,1967,2,83,5,6,4,3,126,2,5,4,None,4.0,-2.0,4,-2,1,1,2,8000.0,3,3,120.0,2,4,4,4,160,1,1,1.0,3,2,80000.0,2000,1,1,2,2,0,1,0,0,0,0,0,0,0,None,0.0,2,1,5,5,5,4,4,3,5,4,4,2,3,-2,1,1,2,3,1990.0,1990.0,2,2,2,5,1,3,1.0,1,4,1,0,1,1,0,0,0,0,0,0,None,29,90,80.0,75,79,80.0,90,90,90,75,4,0,3,1968.0,3.0,1.0,6000.0,1.0,3.0,0.0,0.0,2,5.0,2.0,3,0,1,2,2015/7/21 13:24,2,3,3,-8,3,1,3,3,3,4,3,3,3,3,4,122,2,0.0,0.0,0.0,0.0
3,1943,1,28,5,7,5,2,51,4,4,3,None,4.0,1959.0,4,-2,14,1,2,12000.0,3,3,78.0,2,5,4,4,163,1,1,2.0,4,2,10000.0,6420,2,2,2,2,0,1,0,0,0,0,0,0,0,None,0.0,4,1,5,5,5,5,2,4,5,4,5,1,1,-2,1,1,2,7,1960.0,0.0,2,1,1,5,1,1,0.0,1,3,1,0,0,0,1,0,0,0,0,0,None,10,100,90.0,70,80,80.0,90,90,80,80,4,1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,6.0,1.0,2,1,1,2,2015/7/25 17:33,2,3,3,3,3,2,3,4,3,5,3,3,5,4,3,170,4,0.0,0.0,0.0,0.0
4,1994,1,18,1,1,1,4,36,0,3,12,None,1.0,2014.0,2,1970,6,1,10,-2.0,4,3,70.0,2,4,5,5,165,1,2,3.0,5,-8,200000.0,-1,1,2,2,2,0,1,0,0,0,0,0,0,0,None,0.0,4,3,5,5,1,3,3,2,4,4,3,5,2,1972,4,1,15,1,0.0,0.0,1,3,4,2,5,5,0.0,1,2,2,0,0,0,0,1,0,0,0,0,None,7,50,50.0,50,50,50.0,50,50,50,50,3,1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3,5.0,7.0,4,0,2,3,2015/8/10 9:50,1,4,3,3,3,2,3,3,3,5,5,3,4,3,3,110,6,0.0,0.0,0.0,0.0
